In [ ]:
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import multiprocessing as mp
from os.path import exists
from functools import partial
from tqdm import tqdm
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
def read_df(name):
    if exists('../input/%s.h5' %name):
        df = pd.read_hdf('../input/%s.h5' %name, str(name))
    else:
        df = pd.read_csv('../input/%s.csv' %name)
        df.to_hdf('../input/%s.h5' %name, str(name))
    return df

def rename_columns(df, name):
    for col in df.columns:
        if col != 'SK_ID_CURR':
            df.rename(index=str, columns={col: name+'__'+col}, inplace=True)
    return df

In [ ]:
# Preprocess application_train.csv and application_test.csv
def application_train_test(nan_as_category = False):
    # Read data and merge
    df = read_df('application_train')
    test_df = read_df('application_test')
    if DEBUG:
        df = df[:HEAD]
        test_df = test_df[:HEAD]
    print("Train samples: {}, test samples: {}".format(len(df), len(test_df)))
    df = df.append(test_df).reset_index()
    
    df['CODE_GENDER'].replace('XNA',np.nan, inplace=True) # df = df[df['CODE_GENDER'] != 'XNA']
    df['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True)
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)
    df['NAME_FAMILY_STATUS'].replace('Unknown', np.nan, inplace=True)
    df['ORGANIZATION_TYPE'].replace('XNA', np.nan, inplace=True)
    return df
# Preprocess bureau.csv and bureau_balance.csv
def bureau_and_balance(nan_as_category = True):
    bureau = read_df('bureau')

    bb = read_df('bureau_balance')
    if DEBUG:
        bureau = bureau[:HEAD]
        bb = bb[:HEAD]
    # preprocessing
    bureau['DAYS_CREDIT_ENDDATE'][bureau['DAYS_CREDIT_ENDDATE'] < -40000] = np.nan
    bureau['DAYS_CREDIT_UPDATE'][bureau['DAYS_CREDIT_UPDATE'] < -40000] = np.nan
    bureau['DAYS_ENDDATE_FACT'][bureau['DAYS_ENDDATE_FACT'] < -40000] = np.nan

    # Bureau balance: Perform aggregations
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
    
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])

    # merge bureau_balance.csv with bureau.csv
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
    bureau.drop(['SK_ID_BUREAU'], axis=1, inplace= True)

    del bb, bb_agg
    gc.collect()
    return bureau_agg
    
# Preprocess previous_applications.csv
def previous_applications(nan_as_category = True):
    prev_app = read_df('previous_application')
    if DEBUG:
        prev_app = prev_app[:HEAD]

    #------------------
    # preprocessing
    #------------------
    prev_app['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
    prev_app['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
    prev_app['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
    prev_app['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
    prev_app['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
    return prev_app

# Preprocess POS_CASH_balance.csv
def pos_cash(nan_as_category = True):
    pos_cash = read_df('POS_CASH_balance')
    if DEBUG:
        pos_cash = pos_cash[:HEAD]
        
    #------------------
    # preprocessing
    #------------------
    pos_cash['is_contract_status_completed'] = pos_cash['NAME_CONTRACT_STATUS'] == 'Completed'
    pos_cash['pos_cash_paid_late'] = (pos_cash['SK_DPD'] > 0).astype(int)
    pos_cash['pos_cash_paid_late_with_tolerance'] = (pos_cash['SK_DPD_DEF'] > 0).astype(int)
    return pos_cash
    

In [ ]:
def rename_columns(df, name):
    for col in df.columns:
        if col != 'SK_ID_CURR':
            df.rename(index=str, columns={col: name+'__'+col}, inplace=True)
    return df

@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))
    
DEBUG = False
HEAD = 1000
df = pd.DataFrame()

with timer("Process train/test application"):
    df = application_train_test()
    print("Train/Test application df shape:", df.shape)
    
with timer("Process bureau and bureau_balance"):
    bureau = bureau_and_balance()
    bureau = rename_columns(bureau, 'bureau')
    df = df.merge(bureau, how='left', on=['SK_ID_CURR'])
    print("Train/Test application df shape:", df.shape)
    #del bureau
    gc.collect()
    
with timer("Process previous_applications"):
    prev = previous_applications()
    prev = rename_columns(prev, 'previous_applications')
    df = df.merge(prev, how='left', on=['SK_ID_CURR'])
    print("Train/Test application df shape:", df.shape)
    #del prev
    gc.collect()
    
with timer("Process POS-CASH balance"):
    pos = pos_cash()
    pos = rename_columns(pos, 'POS_CASH')
    print("Pos-cash balance df shape:", pos.shape)
    df = df.merge(pos, how='left', on=['SK_ID_CURR'])
    print("Train/Test application df shape:", df.shape)
    #del pos
    gc.collect()

In [ ]:
feat = [f for f in list(bureau.columns) if 'MONTHS_BALANCE' in f]
feat

In [ ]:
df['bureau__MONTHS_BALANCE_MAX'].value_counts()

In [ ]:
df[df.POS_CASH__MONTHS_BALANCE == -34.0]

In [ ]:
bb

In [ ]:
pd.merge(bureau, bb, how = 'left', on = 'SK_ID_BUREAU')